## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

Import packages

In [1]:
import pandas as pd
import numpy as np

### (a) Download the Anuran Calls (MFCCs) Data Set

In [2]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/Frogs_MFCCs.csv')
X = df.iloc[:,0:22]
y = df.iloc[:,22:25]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
X.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_13,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.156436,0.082245,0.135752,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.254341,0.022786,0.163320,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.237384,0.050791,0.207338,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.317084,-0.011567,0.100413,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.298524,0.037439,0.219153,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244


In [7]:
y.head()

,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraAndre
1,Leptodactylidae,Adenomera,AdenomeraAndre
2,Leptodactylidae,Adenomera,AdenomeraAndre
3,Leptodactylidae,Adenomera,AdenomeraAndre
4,Leptodactylidae,Adenomera,AdenomeraAndre


### (b) Train a classifier for each label

#### (i) Research

 Accuracy score/ Exact match metric calculates subset accuracy meaning the predicted set of labels should exactly match with the true set of labels. This is usually too restrict for multilabel classification. 

 Hamming Loss calculates the fraction of the wrong labels to the total number of labels.  
 
 I compared the results using these two metrics in (ii) - (iv).

#### (ii) Train a SVM for each of the labels

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# default kernel is Guassian/RBF, default decision_function_shape='ovr'
clf = SVC(random_state=0)
selector_fml = GridSearchCV(clf, param_grid={'C': np.logspace(0.1, 1, 15), 'gamma': np.linspace(2.5, 3.5, 10)}, cv=10, n_jobs=-1)
selector_fml.fit(X_train, y_train['Family'])

print(selector_fml.best_params_)
print("best cv score:", selector_fml.best_score_)

{'C': 6.414205312236928, 'gamma': 2.611111111111111}
best cv score: 0.9938448673041119


In [9]:
print("test score for class Family", selector_fml.score(X_test, y_test['Family']))

test score for class Family 0.9939786938397406


In [10]:
selector_gns = GridSearchCV(clf, param_grid={'C': np.logspace(0.1, 1, 15), 'gamma': np.linspace(1, 3, 10)}, cv=10, n_jobs=-1)
selector_gns.fit(X_train, y_train['Genus'])

print(selector_gns.best_params_)
print("best cv score:", selector_gns.best_score_)

{'C': 7.437527275659048, 'gamma': 2.5555555555555554}
best cv score: 0.9910627504812396


In [11]:
print("test score for class Genus", selector_gns.score(X_test, y_test['Genus']))

test score for class Genus 0.9888837424733673


In [12]:
selector_spc = GridSearchCV(clf, param_grid={'C': np.logspace(0.1, 1.5, 15), 'gamma': np.linspace(2.5, 3.5, 10)}, cv=10, n_jobs=-1)
selector_spc.fit(X_train, y_train['Species'])

print(selector_spc.best_params_)
print("best cv score:", selector_spc.best_score_)

{'C': 9.999999999999998, 'gamma': 2.7222222222222223}
best cv score: 0.9910627504812396


In [13]:
print("test score for class Species", selector_spc.score(X_test, y_test['Species']))

test score for class Species 0.9888837424733673


In [25]:
from sklearn.metrics import accuracy_score, hamming_loss
                  
def hamming_score(y_true, y_pred):
    '''
    Compute the hamming score for the multi-label case.
    '''
    # part of the code of this function refers to https://github.com/ruch0401
    missclf_labels = 0
    for truth, pred in zip(y_true, y_pred):
        miss = (truth != pred)
        missclf_labels += np.sum(miss)
    score = 1 - missclf_labels / (y_true.shape[0] * y_true.shape[1])
    
    return score
    
def exact_match_accuracy(y_true, y_pred):
    '''
    Compute the exact match accuracy for the multi-label case.
    '''
    # Check if the shapes of y_true and y_pred are the same
    if y_true.shape != y_pred.shape:
        raise ValueError("Input shapes do not match.")

    num_samples = y_true.shape[0]
    correct_matches = 0

    for i in range(num_samples):
        if np.array_equal(y_true[i], y_pred[i]):
            correct_matches += 1

    accuracy = correct_matches / num_samples
    return accuracy

In [15]:
y_pred_fml = selector_fml.predict(X_test)
y_pred_gns = selector_gns.predict(X_test)
y_pred_spc = selector_spc.predict(X_test)
y_pred = np.column_stack((y_pred_fml, y_pred_gns, y_pred_spc))
y_true = np.array(np.column_stack((y_test['Family'], y_test['Genus'], y_test['Species'])))

print('Hamming score:', hamming_score(y_true, y_pred))
print('Exact match accuracy:', exact_match_accuracy(y_true, y_pred))

Hamming score: 0.9905820595954917
Exact match accuracy: 0.9861046780917091


#### (iii) Repeat 1(b)ii with L1-penalized SVMs

In [16]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf_scaled = make_pipeline(StandardScaler(), LinearSVC(penalty='l1', dual="auto", random_state=0, max_iter=5000))
labels = ['Family', 'Genus', 'Species']

y_pred_l1 = []

for label in labels:
    selector_l1 = GridSearchCV(clf_scaled, param_grid={'linearsvc__C': np.logspace(-1, 1.5, 20)}, cv=10, n_jobs=-1)
    selector_l1.fit(X_train, y_train[label])
    
    y_pred_l1.append(selector_l1.predict(X_test))
    
    print("best params for class", label, ":", selector_l1.best_params_)
    print("best cv score for class", label, ":", selector_l1.best_score_)
    print("test score for class", label, ":", selector_l1.score(X_test, y_test[label]))
    print("---------------------------------------------")
    

best params for class Family : {'linearsvc__C': 0.6158482110660264}
best cv score for class Family : 0.9410264602859038
test score for class Family : 0.9282075034738305
---------------------------------------------


/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250

best params for class Genus : {'linearsvc__C': 9.412049672680666}
best cv score for class Genus : 0.9527422764997319
test score for class Genus : 0.9416396479851783
---------------------------------------------


/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250

best params for class Species : {'linearsvc__C': 1.5283067326587687}
best cv score for class Species : 0.9604839218656316
test score for class Species : 0.9587772116720704
---------------------------------------------


In [17]:
y_pred = np.column_stack(y_pred_l1)

print('Hamming score:', hamming_score(y_true, y_pred))
print('Exact match accuracy:', exact_match_accuracy(y_true, y_pred))

Hamming score: 0.9428747877103597
Exact match accuracy: 0.9129226493747105


#### (iv) Repeat 1(b)iii by using SMOTE or any other method for imbalance

In [19]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
y_pred_sm = []

clf_scaled = make_pipeline(StandardScaler(), LinearSVC(penalty='l1', dual="auto", random_state=0, max_iter=5000))

for label in labels:
    X_res, y_res = sm.fit_resample(X_train, y_train[label])
    selector_l1s = GridSearchCV(clf_scaled, param_grid={'linearsvc__C': np.logspace(-1, 3, 20)}, cv=10, n_jobs=-1)
    selector_l1s.fit(X_res, y_res)
    y_pred_sm.append(selector_l1s.predict(X_test))

    print("best params for class", label, ":", selector_l1s.best_params_)
    print("best cv score for class", label, ":", selector_l1s.best_score_)
    print("test score for class", label, ":", selector_l1s.score(X_test, y_test[label]))
    print("---------------------------------------------")

best params for class Family : {'linearsvc__C': 0.26366508987303583}
best cv score for class Family : 0.9512698538702231
test score for class Family : 0.9087540528022232
---------------------------------------------
best params for class Genus : {'linearsvc__C': 143.8449888287663}
best cv score for class Genus : 0.9573564333615856
test score for class Genus : 0.9018063918480778
---------------------------------------------


/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alicella/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1250

best params for class Species : {'linearsvc__C': 20.6913808111479}
best cv score for class Species : 0.9633428688189621
test score for class Species : 0.9615562760537286
---------------------------------------------


In [20]:
y_pred = np.column_stack(y_pred_sm)

print('Hamming score:', hamming_score(y_true, y_pred))
print('Exact match accuracy:', exact_match_accuracy(y_true, y_pred))

Hamming score: 0.9240389069013433
Exact match accuracy: 0.8550254747568319


Comparison between (ii) - (iv):  
- Hamming score is higher than exact match accuracy for all three classifier sets.
- SVMs without L1 penalty has the highest hamming score and exact match accuracy.
- SMOTE worsens the performance of L1-penalized SVMs, resulting in lower hamming score and exact match accuracy.

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

### (a) Use k-means clustering

In [30]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def get_best_k(X, rs=0):
    '''
    Compute the best k for the KMeans clustering using silhouette score.
    Random state is specified by rs.
    '''
    
    best_k = 2
    best_score = 0

    for k in range(2, 51):
        kmeans = KMeans(n_clusters=k, random_state=rs, n_init=10)
        kmeans.fit(X)
        score = silhouette_score(X, kmeans.labels_)

        if score > best_score:
            best_score = score
            best_k = k
    
    return best_k

### (b) i. Determine which family, genus, species is the majority

In [31]:
def get_true_label_clusters(kmeans):
    """
    return a dictionary matching each cluster label to a numpy array of true labels
    """
    cluster_labels = kmeans.labels_
    unique, counts = np.unique(cluster_labels, return_counts=True)
    # print(np.asarray((unique, counts)).T)

    clusters = {label: [] for label in unique}

    # Iterate over each instance and append true label to the corresponding cluster
    for i, label in enumerate(cluster_labels):
        true_label = y.iloc[i]
        clusters[label].append(true_label)

    # Convert lists to numpy arrays for each cluster
    for label in clusters.keys():
        clusters[label] = np.array(clusters[label])

    return clusters

def get_labels_majority_in_each_cluster(clusters):
    """
    Find the majority class of each label in each cluster.
    """

    cluster_nums = [0, 1, 2, 3]
    true_labels = ['family', 'genus', 'species']

    family_majority = []
    genus_majority = []
    species_majority = []

    for cluster_idx in cluster_nums:
        for true_label_idx in range(len(true_labels)):
            var = clusters[cluster_idx][:,true_label_idx]
            unique_classes, counts = np.unique(var, return_counts=True)
            majority_class = unique_classes[np.argmax(counts)]
            if true_label_idx == 0:
                family_majority.append(majority_class)
            elif true_label_idx == 1:
                genus_majority.append(majority_class)
            else:
                species_majority.append(majority_class)

    result = pd.DataFrame({'cluster': cluster_nums, 'family_majority': family_majority, 'genus_majority': genus_majority, 'species_majority': species_majority})
    
    return result

### (c) Calculate the average Hamming distance, Hamming score, and Hamming loss

In [32]:
def get_hamming_distance(y_true, y_pred):
    '''
    Compute the hamming distance between two multi-label vectors.
    '''
    # if y_true.shape != y_pred.shape:
    #     raise ValueError("Input shapes do not match.")

    num_samples = y_true.shape[0]
    hamming_distance = 0
    
    for i in range(num_samples):
        hamming_distance += np.sum(y_true[i] != y_pred[i])

    return hamming_distance

def hamming_metric(majority_res, clusters):
    hm_dist = 0
    hm_score = 0
    for k in clusters.keys():
        majority = majority_res.loc[majority_res['cluster'] == k].iloc[0:,1:4]
        true_labels = clusters[k]
        
        for y_true in true_labels:
            y_true.resize(1,3)
            hm_dist += get_hamming_distance(y_true, majority.values)
            hm_score += hamming_score(y_true, majority.values)

    avg_hm_dist = hm_dist / len(y)
    avg_hm_score = hm_score / len(y)

    return avg_hm_dist, avg_hm_score
        

In [36]:
def monte_carlo(n):
    hm_dist = []
    hm_loss = []
    hm_score = []

    for i in range(1, n+1):
        print("Monte Carlo Iteration #", i)
        
        best_k = get_best_k(X, rs=i)
        print("Best K:", best_k)

        kmeans = KMeans(n_clusters=best_k, random_state=i, n_init=10).fit(X)
        clusters = get_true_label_clusters(kmeans) 
        majority_res = get_labels_majority_in_each_cluster(clusters)
        print(majority_res)
        
        dist, score = hamming_metric(majority_res, clusters)
        loss = 1 - score
        hm_dist.append(dist)
        hm_loss.append(loss)
        hm_score.append(score)
        print(f"Hamming Distance: {round(dist, 4)}  Hamming Loss: {round(loss, 4)}  Hamming Score: {round(score, 4)}")
        print("---------------------------------------------")

    return hm_dist, hm_loss, hm_score

Do (a)-(c) for 50 Monte-Carlo simulation.   
In each iteration, the majority of the three labels in each cluster is displayed in a table.  
The average Hamming distance, Hamming score and Hamming loss are displayed below the table.

In [37]:
hamming_distances, hamming_losses, hamming_scores = monte_carlo(50)

Monte Carlo Iteration # 1
Best K: 4
   cluster  family_majority genus_majority        species_majority
0        0          Hylidae      Hypsiboas       HypsiboasCordobae
1        1  Leptodactylidae      Adenomera  AdenomeraHylaedactylus
2        2    Dendrobatidae       Ameerega      Ameeregatrivittata
3        3          Hylidae      Hypsiboas    HypsiboasCinerascens
Hamming Distance: 0.6673  Hamming Loss: 0.2224  Hamming Score: 0.7776
---------------------------------------------
Monte Carlo Iteration # 2
Best K: 4
   cluster  family_majority genus_majority        species_majority
0        0  Leptodactylidae      Adenomera  AdenomeraHylaedactylus
1        1  Leptodactylidae      Adenomera          AdenomeraAndre
2        2          Hylidae      Hypsiboas       HypsiboasCordobae
3        3          Hylidae      Hypsiboas       HypsiboasCordobae
Hamming Distance: 0.7354  Hamming Loss: 0.2451  Hamming Score: 0.7549
---------------------------------------------
Monte Carlo Iteration # 3


In [38]:
hm_dist_avg = np.mean(hamming_distances)
hm_dist_std = np.std(hamming_distances)
hm_loss_avg = np.mean(hamming_losses)
hm_loss_std = np.std(hamming_losses)
hm_score_avg = np.mean(hamming_scores)
hm_score_std = np.std(hamming_scores)
hamming_summary = {
    'Hamming Distance': {
        'avg': hm_dist_avg,
        'std_dev': hm_dist_std,
    },
    'Hamming Score': {
        'avg': hm_score_avg,
        'std_dev': hm_score_std,
    },
    'Hamming Loss': {
        'avg': hm_loss_avg,
        'std_dev': hm_loss_std
    }
}
result_summary = pd.DataFrame(hamming_summary)
result_summary

,Hamming Distance,Hamming Score,Hamming Loss
avg,0.671889,0.776037,0.223963
std_dev,0.016807,0.005602,0.005602


## Reference

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html  
LinearSVC: Similar to SVC with parameter kernel=’linear’, but implemented in terms of liblinear rather than libsvm, so it has more flexibility in the choice of penalties and loss functions and should scale better to large numbers of samples.

Hamming score for binary multilabel problem: https://hasty.ai/docs/mp-wiki/metrics/hamming-score  
